In [57]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

/Users/hsh/dev_project/sb-fsts


In [58]:
from app.utils.crud_sql import SQLExecutor
from app.utils.database import get_db, get_db_session

sql_executor = SQLExecutor()

In [4]:
query = """
        SELECT 
            *
        FROM 
            fsts.test_table
    """

params = {}

with get_db_session() as db:
    result = sql_executor.execute_select(db, query, params)

print(result)

[{'id': 'id3', 'name': 'name3'}, {'id': 'id5', 'name': 'name5'}, {'id': 'id6', 'name': 'name6'}, {'id': 'id7', 'name': 'name7'}, {'id': 'id1', 'name': 'updated_name1'}, {'id': 'id2', 'name': 'upserted_name2'}, {'id': 'id4', 'name': 'upserted_name4'}]


In [ ]:
# RETURNING * 꼭 붙여야 함.
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

id = 'id4'
name = 'name4'

params = {
    "id": id,
    "name": name
}

with get_db_session() as db:
    result = sql_executor.execute_insert(db, query, params)

In [ ]:
#여러 데이터를 한번에 삽입하고 싶을 때
# 여러 데이터 정의
data = [
    {"id": "id5", "name": "name5"},
    {"id": "id6", "name": "name6"},
    {"id": "id7", "name": "name7"}
]

# 다중 데이터 삽입 쿼리
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

# 데이터 삽입 및 결과 저장
results = []
with get_db_session() as db:
    for row in data:
        result = sql_executor.execute_insert(db, query, row)
        results.append(result)

print("삽입된 데이터:")
print(results)

In [6]:
# UPDATE 테스트
query_update = """
    UPDATE fsts.test_table
    SET name = :name
    WHERE id = :id
    RETURNING *;
"""
#:는 파라미터 바인딩 형식

params_update = {
    "id": "id2",
    "name": "HONG"
}

with get_db_session() as db:
    result_update = sql_executor.execute_update(db, query_update, params_update)

Update succeeded: [{'id': 'id2', 'name': 'HONG'}]


# DynamoDB

In [22]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, ListAttribute, MapAttribute
from datetime import datetime
import time

# 매매 이력 (trading_history)에서 사용될 속성 정의
class TradeHistory(MapAttribute):
    trading_logic = UnicodeAttribute()
    symbol = UnicodeAttribute()
    symbol_name = UnicodeAttribute()
    position = UnicodeAttribute()
    price = NumberAttribute()
    quantity = NumberAttribute()
    created_at = NumberAttribute()
    updated_at = UnicodeAttribute(null=True)
    data_type = UnicodeAttribute()

# DynamoDB 테이블 모델 정의
class TradingBot(Model):
    class Meta:
        table_name = "fsts-trading-bot"
        region = "ap-northeast-2"

    trading_bot_name = UnicodeAttribute(hash_key=True)  # PK 설정
    created_at = NumberAttribute()
    trading_history = ListAttribute(of=TradeHistory)  # 리스트 저장 가능

# 테이블 생성 (최초 1회 실행)
# if not TradingBot.exists():
#     TradingBot.create_table(read_capacity_units=5, write_capacity_units=5, wait=True)

In [45]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute
import time
import uuid

class TradingHistory(Model):
    class Meta:
        table_name = "fsts-trading-history"
        region = "ap-northeast-2"

    trading_bot_name = UnicodeAttribute(hash_key=True)  # ✅ PK
    created_at = NumberAttribute(range_key=True)  # ✅ SK (밀리세컨드 단위)
    updated_at = UnicodeAttribute(null=True)
    symbol = UnicodeAttribute()
    position = UnicodeAttribute()
    price = NumberAttribute()
    quantity = NumberAttribute()
    data_type = UnicodeAttribute()

In [40]:
from pynamodb.transactions import TransactWrite
from pynamodb.connection import Connection
from botocore.exceptions import ClientError

def add_trade(trading_bot_name, symbol, position, price, quantity, data_type):
    max_retries = 3  # 최대 재시도 횟수
    retry_count = 0

    while retry_count < max_retries:
        try:
            created_at = int(time.time() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

            new_trade = TradingHistory(
                trading_bot_name=trading_bot_name,
                created_at=created_at,
                updated_at=None,
                symbol=symbol,
                position=position,
                price=price,
                quantity=quantity,
                data_type=data_type
            )

            connection = Connection()

            with TransactWrite(connection=connection) as transaction:
                transaction.save(new_trade, condition=(TradingHistory.created_at.does_not_exist()))
                print(f"✅ 트랜잭션 성공: {created_at}")
                return True  # 성공적으로 저장되면 종료

        except ClientError as e:
            if e.response["Error"]["Code"] == "TransactionCanceledException":
                print("❌ 중복된 created_at 감지! 새로운 값으로 재시도...")
                retry_count += 1
            else:
                raise  # 다른 에러 발생 시 예외 던지기

    print("🚨 최대 재시도 횟수 초과! 거래 저장 실패")
    return False

In [61]:
def execute_save(data_model):
        max_retries = 3  # 최대 재시도 횟수
        retry_count = 0

        while retry_count < max_retries:
            try:
                connection = Connection()

                created_at = int(time.time() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

                with TransactWrite(connection=connection) as transaction:
                    # transaction.save(data_model, condition=(TradingHistory.created_at.does_not_exist()))
                    transaction.save(data_model, condition=(type(data_model).created_at.does_not_exist()))
                    print(f"✅ 트랜잭션 성공: {created_at}")
                    return True  # 성공적으로 저장되면 종료

            except ClientError as e:
                if e.response["Error"]["Code"] == "TransactionCanceledException":
                    print("❌ 중복된 created_at 감지! 새로운 값으로 재시도...")
                    retry_count += 1
                else:
                    raise  # 다른 에러 발생 시 예외 던지기

        print("🚨 최대 재시도 횟수 초과! 거래 저장 실패")
        return False

In [42]:
add_trade(
    trading_bot_name="MyTradingBot_1234",
    symbol="AAPL",
    position="BUY",
    price=200,
    quantity=1,
    data_type="test"
)

✅ 트랜잭션 성공: 1740408172439


True

In [65]:
from app.utils.dynamodb.model.trading_history_model import TradingHistory
from pytz import timezone


kst = timezone("Asia/Seoul")
# 현재 시간을 KST로 변환
current_time = datetime.now(kst)
created_at = int(current_time.timestamp() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

model = TradingHistory(
    trading_bot_name='123',
    created_at=created_at,
    updated_at=None,
    trading_logic='123',
    trade_date=created_at,
    symbol='123',
    symbol_name='123',
    position='123',
    price=float(12.2),
    quantity=float(12.2),
    data_type='test'
)
execute_save(model)

✅ 트랜잭션 성공: 1741075189398


True

### 데이터 조회

In [53]:
# 특정 trading_bot_name의 데이터 조회
history = TradingHistory.scan()

for trade in history:
    print(f"- Symbol: {trade.symbol}, Price: {trade.price}, Quantity: {trade.quantity}, Created At: {trade.created_at}")

- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740407407897
- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740408172439


In [52]:
# 특정 trading_bot_name의 데이터 조회
history = TradingHistory.query("MyTradingBot_1234")

for trade in history:
    print(f"- Symbol: {trade.symbol}, Price: {trade.price}, Quantity: {trade.quantity}, Created At: {trade.created_at}")

- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740407407897
- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740408172439
